## 算子

**狭义的算子（Kernel）**：对张量 Tensor 执行的基本操作集合，包括四则运算，数学函数，甚至是对张量元数据的修改，如维度压缩（Squeeze），维度修改（reshape）等。

**广义的算子（Function）**：AI 框架中对算子模块的具体实现，涉及到调度模块，Kernel 模块，求导模块以及代码自动生成模块。

### 算子间的调度

无论是大模型还是传统的神经网络模型，实际上最后执行都会落在单台设备环境上执行对应的算子。对单设备执行环境，制约计算图中节点调度执行的关键因素是节点之间的数据流依赖和具体的算子。

### 算子并发调度
单设备算子间使用单线程管理先进先出队列进行调度，这种方式是直接也是最原始的。实际 AI 框架会根据计算图，找到相互独立的算子进行并发调度，提高计算的并行性。

![img.png](https://github.com/chenzomi12/AISystem/raw/main/05Framework/03DataFlow/images/forward_mode03.png)
![img.png](https://github.com/chenzomi12/AISystem/raw/main/05Framework/03DataFlow/images/forward_mode02.png)

非常依赖于计算图能够准确的描述了算子之间的依赖关系，通过后端编译优化功能或者后端编译优化的 Pass，提供并发执行队列的调度操作。


在手机端侧异构计算环境中，主要存在 CPU、GPU 以及 NPU 等多种异构的计算 IP，因此一张计算图可以由运行在不同计算 IP 的算子组成为异构计算图,

异构计算图的优点在：：
1）异构硬件加速，将特定的计算放置到合适的硬件上执行；

2）算子间的并发执行，从计算图上可知，没有依赖关系的算子或者子图，逻辑上可以被 AI 框架并发调用。

目前采用算子异构调度的方式作为推理引擎的新增特性比较多，主要原因在于：1）调度逻辑复杂，程序控制实现起来并不简单，即使自动化方式只能针对部分神经网络模型；2）大部分神经网络模型的结构，仍然以高度串行为主，上下节点之间的依赖较重；3）异构调度涉及到不同 IP 计算结果之间的通信，通信的开销往往大于计算的开销。导致计算需要等待数据的同步与传输。

## 图优化

计算图的执行方式，可以分为两种模式：
1）逐算子下发执行的交互式方式，如 PyTroch 框架；
2）以及整个计算图或者部分子图一次性下发到硬件进行执行，如 TensorFlow 和 MindSpore。无论采用哪种模式，其大致架构如下所示。
![image.png](./img/framework05.png)

## 图调度
它主要负责对计算图中的各个算子（操作）进行合理安排，以提升计算效率和资源利用率。

图调度的作用

- 优化计算顺序：计算图中的算子存在先后执行顺序，合理的调度能确保算子按照最优顺序执行，减少不必要的等待时间。
- 资源分配与管理：调度器要为每个算子分配合适的计算资源（如 CPU 或 GPU 内存），避免资源竞争和浪费。比如在多 GPU 环境下，调度器会将不同的算子分配到不同的 GPU 上并行执行
- 减少内存占用：通过合理安排算子的执行顺序和内存分配，调度器可以尽量减少中间结果的内存占用。例如，某些中间结果在后续计算中不再使用时，调度器可以及时释放其占用的内存。

### 静态调度
- 静态调度：在计算图构建完成后，预先确定好所有算子的执行顺序和资源分配。这种调度方式适用于计算图结构固定的情况，如训练过程中每次迭代的计算图都相同。静态调度可以进行全局优化，提高执行效率。
- 动态调度：在计算图执行过程中实时决定算子的执行顺序和资源分配。这种调度方式更灵活，适用于计算图结构动态变化的场景，如模型中包含控制流（如条件判断、循环）的情况。


## 图执行

AI 框架生成计算图后，经过图调度模块对进行图进行标记，计算图已经准备好被实际的硬件执行，根据硬件能力的差异，可以将异构计算图的执行分为三种模式：

- 单算子执行、主要针对 CPU 和 GPU 的场景，计算图中的算子按照输入和输出的依赖关系被逐个调度与执行

- 整图下沉执行，整图下沉执行模式主要是针对 DSA 架构的 AI 芯片而言，其主要的优势是能够将整个计算图一次性下发到设备上，无需借助 CPU 的调度能力而独立完成计算图中所有算子的调度与执行，减少了主机和 AI 芯片的交互次数，借助 AI 芯片并行加速能力，提高计算效率和性能

3）图切分到多设备执行、图切分与多设备执行的方式是面向大规模计算场景的，如现在很火的大模型。由于计算图自身表达的灵活性，对于复杂场景的计算图在 AI 芯片上进行整图下沉执行的效率不一定能达到最优，或者在单个 AI 芯片上不能完整放下一张计算图。因此可以将计算图进行拆分，把大模型产生的计算图分别放在不同的 AI 加速芯片上面。此外，对于 AI 芯片执行效率低下的部分分离出来，交给 CPU 处理，将更适合 AI 芯片的子图下沉到 AI 芯片进行计算，这样可以兼顾性能和灵活性两方面。

